In [132]:
#Importing required packages.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline
from keras import models
wine = pd.read_csv('winequality-red.csv')

drop_model = models.Sequential()
drop_model.add(layers.Dense(64, activation='relu', input_shape=(NB_WORDS,)))
drop_model.add(layers.Dropout(0.5))
drop_model.add(layers.Dense(64, activation='relu'))
drop_model.add(layers.Dropout(0.5))
drop_model.add(layers.Dense(3, activation='softmax'))
drop_model.name = 'Dropout layers model'
drop_history = deep_model(drop_model, X_train_rest, y_train_rest, X_valid, y_valid)
drop_min = optimal_epoch(drop_history)
eval_metric(drop_model, drop_history, 'loss')

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
wine.head()

In [ ]:
wine.info()

In [ ]:
fig = plt.figure(figsize = (10,6))
wine.plot(kind='bar', x='quality', y='fixed acidity')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'fixed acidity', data = wine)

In [ ]:
wine.plot(kind = 'bar', x = 'quality', y = 'volatile acidity')
plt.show()

In [ ]:
fig2 = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'volatile acidity', data = wine)

In [ ]:
reviews = []
for i in wine['quality']:
    if i >= 2 and i < 6.5:
        reviews.append('0')
    elif i >= 6.5 and i <= 8:
        reviews.append('1')
wine['Reviews'] = reviews

In [ ]:
wine.columns


In [ ]:
from collections import Counter
Counter(wine['Reviews'])

In [ ]:
label_quality = LabelEncoder()

In [ ]:
wine['Reviews'] = label_quality.fit_transform(wine['Reviews'])

In [ ]:
sns.countplot(wine['Reviews'])

In [ ]:
X = wine.drop('Reviews', axis = 1)
X = wine.drop('quality', axis = 1)
y = wine['Reviews']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 20)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(float))
X_test = sc.fit_transform(X_test.astype(float))

In [ ]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, pred_rfc))

In [ ]:
print(classification_report(y_test, pred_rfc))

In [ ]:
print(confusion_matrix(y_test, pred_rfc))

In [ ]:
sgd = SGDClassifier(penalty=None)
sgd.fit(X_train, y_train)
pred_sgd = sgd.predict(X_test)

In [ ]:
print(classification_report(y_test, pred_sgd))

In [ ]:
print(confusion_matrix(y_test, pred_sgd))